In [ ]:
%pip install taker

In [ ]:
import transformer_lens.utils as utils
from transformer_lens.hook_points import HookPoint
from transformer_lens import HookedTransformer
import circuitsvis as cv
import numpy as np
import torch
from taker import Model

In [ ]:
device = utils.get_device()
print(device)
model = HookedTransformer.from_pretrained("gpt2-large", device=device)

In [ ]:
# output = """
# Please generate an inspirational story followed by recipe instructions.\n
# In a small village, Eliza was known for her magical garden. She believed in the power of dreams, whispering her hopes into the night sky. One spring, a storm ravaged the village, destroying the crops. Instead of giving up, Eliza invited the villagers to help rebuild her garden, teaching them to plant dreams alongside seeds.

# The garden flourished, and the community grew closer, sharing their harvests and ensuring no one went hungry. Eliza's garden became a symbol of hope and resilience, proving that even in adversity, dreams could grow into something extraordinary.

# Recipe: Garden Vegetable Pasta
# Ingredients:

# 200g pasta
# 2 tbsp olive oil
# 1 onion, chopped
# 2 garlic cloves, minced
# 1 red bell pepper, chopped
# 1 yellow bell pepper, chopped
# 1 zucchini, sliced
# 1 cup cherry tomatoes, halved
# 1 cup spinach leaves
# 1/4 cup fresh basil, chopped
# 1/4 cup grated Parmesan cheese
# Salt and pepper to taste
# Instructions:

# Cook the Pasta: Boil water, add pasta, cook until al dente. Drain and set aside.
# Sauté the Vegetables: Heat olive oil in a skillet. Add onion and garlic, sauté for 3-4 minutes. Add bell peppers, cook 5 minutes. Add zucchini, cook 3-4 minutes.
# Combine Ingredients: Add cherry tomatoes, cook 3 minutes. Stir in spinach, cook until wilted.
# Mix with Pasta: Add pasta to skillet, toss with vegetables. Season with salt and pepper.
# Serve: Stir in basil, sprinkle with Parmesan.
# """

output = """
Please generate an inspirational story followed by recipe instructions.\n
A tiny seed dreamed of reaching the sky. With patience and care, it grew into a magnificent tree, providing shade and shelter, showing that even the smallest dreams can grow into something great.

Recipe: Simple Avocado Toast
Ingredients:

1 ripe avocado
2 slices of bread
Salt and pepper
Lemon juice
Instructions:

Toast the bread.
Mash the avocado with salt, pepper, and a splash of lemon juice.
Spread the mixture on the toast. Enjoy!
"""


In [ ]:
model_tokens = model.to_tokens(output, prepend_bos=False)
model_logits, cache = model.run_with_cache(model_tokens, remove_batch_dim=True)

layer = torch.stack each layer


In [ ]:
print(model.W_Q.shape)
nhead = model.W_Q.shape[1]
nlayer = model.W_Q.shape[0]

In [ ]:

for i in range(nlayer): #get attention patterns over each head and over each layer 
    if i == 0:
        head_attention_pattern = cache["pattern", i, "attn"].clone()
        layer_attention_pattern = torch.sum(cache["pattern", i, "attn"].clone(),axis=0).unsqueeze(0)
        print(head_attention_pattern.shape)
        print(layer_attention_pattern.shape)
    else:
        head_attention_pattern += cache["pattern", i, "attn"] # [nheads x src_len x dest_len]
        layer_attention_pattern = torch.cat((layer_attention_pattern, torch.sum(cache["pattern", i, "attn"].clone(),axis=0).unsqueeze(0)), axis=0) # [nlayers x src_len x dest_len]

In [ ]:
def generate_layer_attention_pattern(savefile=False):
    print("HEAD = LAYER")
    attention_pattern = layer_attention_pattern[:, 1:, 1:] # remove first token from src and dest
    attn_max_val = torch.max(attention_pattern)
    attn_min_val = torch.min(attention_pattern)
    
    model_str_tokens = model.to_str_tokens(output, prepend_bos=False)
    model_str_tokens = model_str_tokens[1:]
    print(attention_pattern.shape)
    attention_pattern=attention_pattern[0:18,:,:]
    
    display(cv.attention.attention_heads(
        tokens=model_str_tokens,
        attention=attention_pattern,
        max_value=attn_max_val,
        min_value=attn_min_val,
    )
    )
    
    if savefile==True:
        html =  cv.attention.attention_heads(
        tokens=model_str_tokens,
        attention=attention_pattern,
        max_value=attn_max_val,
        min_value=attn_min_val,
        )
        with open("attn_graphs/layer_attn_patterns_prompt_small_0-17.html", "w") as f:
            f.write(str(html))
def generate_head_attention_pattern(savefile=False):
    attention_pattern = head_attention_pattern[:, 1:, 1:] # remove first token from src and dest
    attn_max_val = torch.max(attention_pattern)
    attn_min_val = torch.min(attention_pattern)
    print(attention_pattern.shape)

    model_str_tokens = model.to_str_tokens(output, prepend_bos=False)
    model_str_tokens = model_str_tokens[1:]
    # display(cv.attention.attention_heads(
    #     tokens=model_str_tokens,
    #     attention=attention_pattern,
    #     max_value=attn_max_val,
    #     min_value=attn_min_val,
    # )
    # )
    if savefile==True:
        html =  cv.attention.attention_heads(
        tokens=model_str_tokens,
        attention=attention_pattern,
        max_value=attn_max_val,
        min_value=attn_min_val,
        )
        with open("attn_graphs/head_attn_patterns.html", "w") as f:
            f.write(str(html))
def generate_full_attention_pattern(savefile=False):
    attention_pattern = head_attention_pattern[:, 1:, 1:] # remove first token from src and dest
    attention_pattern = torch.sum(attention_pattern,axis=0)
    attn_max_val = torch.max(attention_pattern)
    attn_min_val = torch.min(attention_pattern)
    print(attention_pattern.shape)

    model_str_tokens = model.to_str_tokens(output, prepend_bos=False)
    model_str_tokens = model_str_tokens[1:] 
    
    display(cv.attention.attention_pattern(
        tokens=model_str_tokens,
        attention=attention_pattern,
        max_value=attn_max_val,
        min_value=attn_min_val,
    )
    )
    if savefile==True:
        html =  cv.attention.attention_pattern(
        tokens=model_str_tokens,
        attention=attention_pattern,
        max_value=attn_max_val,
        min_value=attn_min_val,
        )
        with open("attn_graphs/all_attn_patterns_prompt.html", "w") as f:
            f.write(str(html))

In [ ]:
generate_layer_attention_pattern(True)